Подготовка данных

In [ ]:
import os
import time
import copy
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL

import torch
import torch.nn as nn
import torch.optim as optim

from tabulate import tabulate
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from skimage import io

In [ ]:
data_root = '/kaggle/input/airbus-ship-detection'
train_root = f'{data_root}/train_v2'
validation_root = f'{data_root}/test_v2'

In [ ]:
df_for_count = pd.read_csv(f'{data_root}/train_ship_segmentations_v2.csv')
print(f"Count images in segmentation file {df_for_count['ImageId'].value_counts().shape[0]}")

In [ ]:
with_ships = df_for_count.dropna()
with_ships = with_ships.groupby('ImageId').size().reset_index(name='counts')
without_ships = df_for_count[df_for_count['EncodedPixels'].isna()]

In [ ]:
plt.figure(figsize=(15, 6))
plt.subplot(1,2,1)
plt.bar(['With ships','Without ships'], [len(with_ships),len(without_ships)], color = ['blue','green'])
plt.ylabel('Number of images')
plt.title('Train availability comparison')
plt.grid()

In [ ]:
counts = with_ships['counts'].value_counts(sort=False)
print(tabulate([(i, counts[i]) for i in range(1,16)], headers=['num of ships','num of images']))

In [ ]:
w, h = 3, 3
load_img = lambda filename: np.array(PIL.Image.open(f"{train_root}/{filename}").resize((200, 200)))
_, axes_list = plt.subplots(h, w, figsize=(2*w, 2*h))

for axes in axes_list:
    for ax in axes:
        ax.axis('off')
        img = np.random.choice(os.listdir(train_root))
        ax.imshow(load_img(img))
        ax.set_title(img)

DataLoader

In [ ]:
def make_classes(csv_file, start, lenght):
    img_labels = pd.read_csv(csv_file, index_col=0)
    img_labels['Class'] = img_labels['EncodedPixels'].notnull()
    img_labels = img_labels.groupby("ImageId").agg(['first'])
    img_labels = img_labels.drop(['EncodedPixels'], axis = 1)
    img_labels.columns = ['Class']
    return img_labels[start: start + lenght + 1]

class ShipsDataset(Dataset):
    def __init__(self, csv_file, data_folder, start, lenght, transform=None):
        self.labels = make_classes(csv_file, start, lenght)
        self.transform = transform
        self.data_folder = data_folder
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.data_folder,
                            self.labels.index[idx])
        image = io.imread(img_name)
        label = self.labels.iloc[idx][0]
        label = label.astype('int')
        if self.transform:
            image = self.transform(image)
        return image, label

Данные для моделки

In [ ]:
num_classes = 2
batch_size = 30
num_epochs = 4
feature_extract = True

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0
            for (inputs, labels) in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
        print()
    time_elapsed = time.time() - since
    print(f'Training time: {time_elapsed // 60}m {time_elapsed % 60}s')
    print(f'Best score: {best_acc}')
    model.load_state_dict(best_model_wts)
    
    return model, val_acc_history

Делаем модельку

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(num_classes, feature_extract, use_pretrained=True):
    model_ft = models.densenet121(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Linear(num_ftrs, num_classes)
    input_size = 224
    return model_ft, input_size

model_ft, input_size = initialize_model(num_classes, feature_extract, use_pretrained=True)

In [ ]:
data_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

data = {
    'train':
    ShipsDataset(f'{data_root}/train_ship_segmentations_v2.csv', train_root, 0, 1500,  transform=data_transforms),
    'val':
    ShipsDataset(f'{data_root}/train_ship_segmentations_v2.csv', train_root, 1500, 500,  transform=data_transforms)

}

dataloaders_dict = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'val': DataLoader(data['val'], batch_size=batch_size, shuffle=True)
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_ft = model_ft.to(device)
params_to_update = model_ft.parameters()
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Тренируем модельку

In [ ]:
criterion = nn.CrossEntropyLoss()
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

Сохраняем

In [ ]:
save_path = './ships_model.pth'
torch.save(model_ft.state_dict(), save_path)

Тестируем

In [ ]:
model = initialize_model(num_classes, feature_extract, use_pretrained=True)[0]
model_dict = torch.load('ships_model.pth')
model.load_state_dict(model_dict)
model.eval()

filenames = os.listdir(validation_root)
preds_list = []
images_list = []

for i in range(16):
    id = random.randint(0, 2024)
    input_image = PIL.Image.open(f'{validation_root}/{filenames[id]}')
    
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)

    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    _, preds = torch.max(output, 1)

    preds_list.append(preds[0])
    images_list.append(input_image)

w, h = 4, 4
_, axes_list = plt.subplots(h, w, figsize=(2*w, 2*h))
photo_index = 0
for axes in axes_list:
    for ax in axes:
        ax.axis('off')
        ax.imshow(images_list[photo_index])
        ax.set_title(f'{preds_list[photo_index] == 1}')
        photo_index += 1